In [1]:
// https://github.com/alexarchambault/jupyter-scala
import $ivy.`org.slf4j:slf4j-nop:1.7.12`
import $ivy.`org.apache.spark::spark-sql:2.1.0`
import $ivy.`org.apache.spark::spark-mllib:2.1.0`
import $ivy.`org.jupyter-scala::spark:0.4.0-RC3`
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import jupyter.spark._ // this might obviate the need for `import sqlContext.implicits._` below

import $ivy.$                           

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                                   

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.rdd.RDD

import jupyter.spark._ // this might obviate the need for `import sqlContext.implicits._` below

In [2]:
@transient val sparkConf = new SparkConf().setAppName("20170216_Spark_MLLib_Recommender_Scala").setMaster("local[2]")
@transient val sc = new JupyterSparkContext(sparkConf)
val sqlContext = new SQLContext(sc)
//import sqlContext.implicits._ // this causes an error in the next frame:
sc.version                       // "cmd2.sc:190: object Helper is not a member of package $sess"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/fred/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-nop/1.7.12/slf4j-nop-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/fred/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.helpers.NOPLoggerFactory]
log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


sparkConf: SparkConf = org.apache.spark.SparkConf@1e5fccd5
sc: JupyterSparkContext = jupyter.spark.JupyterSparkContext@746d16bc
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@3227a364
res1_3: String = "2.1.0"

In [3]:
// load the raw ratings data
val datasets_path = "/home/fred/data/amazon"
// String joinedPath = new File(path1, path2).toString();
val small_ratings_file = new java.io.File(datasets_path, "ratings_Baby.csv").toString
val small_ratings_raw_data = sc.textFile(small_ratings_file)
//val small_ratings_raw_data_header = small_ratings_raw_data.take(1)(0) // ain't no header in my data

datasets_path: String = "/home/fred/data/amazon"
small_ratings_file: String = "/home/fred/data/amazon/ratings_Baby.csv"
small_ratings_raw_data: RDD[String] = /home/fred/data/amazon/ratings_Baby.csv MapPartitionsRDD[1] at textFile at cmd2.sc:4

In [4]:
// w/out this the following error: "cmd3.sc:15: value toDF is not a member of org.apache.spark.rdd.RDD[Helper.this.Rating]"
import sqlContext.implicits._

import sqlContext.implicits._

In [5]:
// using default 'user' and 'item' column names as used by ALS.getUserCol and getItemCol
case class Rating(user: Int, item: Int, rating: Double)
//def str2int(s: String): Int = { math.abs(s.hashCode) % math.pow(10, 8).toInt }
object Rating {
    def apply(tokens: Array[String]) = new Rating(tokens(0).hashCode, tokens(1).hashCode, tokens(2).toDouble)
}

// see http://stackoverflow.com/questions/34964565/how-to-create-a-dataset-from-custom-class-person
// for why this is necessary (see 'Spark Notebook' comment)
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

// parse the raw data into a new RDD
val small_ratings_data = small_ratings_raw_data
    //.filter((line: String) => line != small_ratings_raw_data_header)
    .map(_.split(","))
    .map(Rating(_)).toDF.as[Rating].cache

// TODO: implement implicit preferences perhaps? like here:
// https://github.com/eBay/Spark/blob/master/examples/src/main/scala/org/apache/spark/examples/mllib/MovieLensALS.scala

defined class Rating
defined object Rating
small_ratings_data: Dataset[Rating] = [user: int, item: int ... 1 more field]

In [6]:
// for illustrative purposes, take the first few lines of our RDD to see the result
small_ratings_data.take(3)

res5: Array[Rating] = Array(
  Rating(-1613580310, -725463517, 5.0),
  Rating(1185180308, -725463263, 5.0),
  Rating(1941063073, -725461590, 4.0)
)

In [7]:
val nratings = small_ratings_data.count() // 915446
val nusers = small_ratings_data.map(_.user).distinct().count() // 531854
val nitems = small_ratings_data.map(_.item).distinct().count() // 64391

nratings: Long = 915446L
nusers: Long = 531854L
nitems: Long = 64391L

In [8]:
// split it into train, validation, and test datasets
val Array(training, validation_, test_) = small_ratings_data.randomSplit(Array(0.6, 0.2, 0.2), seed=0)
println(s"Before user filter dataset counts (training, validation, test) = (${training.count()}, ${validation_.count()}, ${test_.count()})")

// remove users from validation and test datasets that aren't in training, else suffer NaNs
// per here: https://issues.apache.org/jira/browse/SPARK-14489
val trusers_ = training.map(_.user).collect.toSet
val tritems_ = training.map(_.item).collect.toSet
val trusers = sc.broadcast(trusers_) // http://stackoverflow.com/questions/26214112/filter-based-on-another-rdd-in-spark
val tritems = sc.broadcast(tritems_)
val validation = validation_.filter(x => trusers.value.contains(x.user) && tritems.value.contains(x.item))
val test = test_.filter(x => trusers.value.contains(x.user) && tritems.value.contains(x.item))
println(s"After user filter dataset counts (training, validation, test) = (${training.count()}, ${validation.count()}, ${test.count()})")
println(s"Valdation set size reduced by ${100 - 100 * validation.count() / validation_.count()}%")
training.describe().show
validation.describe().show
test.describe().show

Before user filter dataset counts (training, validation, test) = (548570, 183503, 183373)
After user filter dataset counts (training, validation, test) = (548570, 87141, 86807)


Valdation set size reduced by 53%
+-------+--------------------+--------------------+------------------+
|summary|                user|                item|            rating|
+-------+--------------------+--------------------+------------------+
|  count|              548570|              548570|            548570|
|   mean|  -6969681.643890479|-3.856756676566637E8| 4.116803689592942|
| stddev|1.2403181335590982E9|1.2912695594648366E9|1.2901576455818002|
|    min|         -2147446657|         -2147455739|               1.0|
|    max|          2147479032|          2147211727|               5.0|
+-------+--------------------+--------------------+------------------+



+-------+-------------------+--------------------+-----------------+
|summary|               user|                item|           rating|
+-------+-------------------+--------------------+-----------------+
|  count|              87141|               87141|            87141|
|   mean| -8592992.062496414|-3.471510736339151E8|4.178756268576215|
| stddev|1.241249516808962E9|1.2872958496973834E9|1.203833071805968|
|    min|        -2147402751|         -2147455739|              1.0|
|    max|         2147354236|          2146960956|              5.0|
+-------+-------------------+--------------------+-----------------+

+-------+--------------------+--------------------+------------------+
|summary|                user|                item|            rating|
+-------+--------------------+--------------------+------------------+
|  count|               86807|               86807|             86807|
|   mean|  -8931683.362678126|-3.54418087480617...| 4.182911516352368|
| stddev|1.246027454530

training: Dataset[Rating] = [user: int, item: int ... 1 more field]
validation_: Dataset[Rating] = [user: int, item: int ... 1 more field]
test_: Dataset[Rating] = [user: int, item: int ... 1 more field]
trusers_: Set[Int] = Set(
  -1106602373,
  -1959073589,
  1913896141,
  -1686757815,
  -912333438,
  -1753452537,
  930611718,
  414560163,
  -1645973862,
  -1892452500,
  -1492970957,
...
tritems_: Set[Int] = Set(
  -1792610910,
  2136598920,
  -1797079704,
  1349788168,
  -2110032324,
  -1411554330,
  -174974075,
  -510289111,
  -1427517807,
  -1212341624,
  691349445,
...
trusers: broadcast.Broadcast[Set[Int]] = Broadcast(18)
tritems: broadcast.Broadcast[Set[Int]] = Broadcast(19)
validation: Dataset[Rating] = [user: int, item: int ... 1 more field]
test: Dataset[Rating] = [user: int, item: int ... 1 more field]

In [9]:
// proceed with the training phase
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import org.apache.spark.ml.evaluation.RegressionEvaluator
// old API: https://spark.apache.org/docs/2.1.0/api/scala/index.html#org.apache.spark.mllib.recommendation.ALS
// new API: https://spark.apache.org/docs/2.1.0/api/scala/index.html#org.apache.spark.ml.recommendation.ALS

val seed = 5
val niters = 10
val regularization_parameters = Array(0.3, 0.6)
val ranks = Array(18, 22)
val tolerance = 0.02

// https://spark.apache.org/docs/latest/ml-collaborative-filtering.html
def rmse(model: ALSModel, data: Dataset[Rating]) = {
    val predictions = model.transform(data)
    val evaluator = new RegressionEvaluator()
        .setMetricName("rmse")
        .setLabelCol("rating")
        .setPredictionCol("prediction")
    val error = evaluator.evaluate(predictions)
    (error, predictions)
}

var minerr = Double.PositiveInfinity
var best: (Double, Int, ALSModel) = (-1.0, -1, null) // "class Helper needs to be abstract, since variable best is not defined"
for(rp <- regularization_parameters; rank <- ranks) {
    
    val als = new ALS()
        .setRank(rank) // number of latent factors in the model
        .setMaxIter(niters) // formerly setIterations
        .setRegParam(rp) // formerly setLambda (lambda specifies the regularization parameter in ALS)
        .setImplicitPrefs(false) // implicit or explicit recommendations?
        //.setNumUserBlocks(-1) // IllegalArgumentException: als_2b89f15f5bde parameter numUserBlocks given invalid value -1
        //.setNumItemBlocks(-1) // IllegalArgumentException: als_e9bc5c270983 parameter numItemBlocks given invalid value -1
    
    val model = als.fit(training)
    
    val terr = rmse(model, training)._1
    val verr = rmse(model, validation)._1
    println(s"For rp/rank ${(rp, rank)} the training RMSE is $terr and validation RMSE is $verr")
    if (verr < minerr) {
        minerr = verr
        best = (rp, rank, model)
    }
}

println(s"The best model was trained with rp/rank $best")

For rp/rank (0.3,18) the training RMSE is 0.4077053116456953 and validation RMSE is 2.13740144296016


For rp/rank (0.3,22) the training RMSE is 0.4060075868931709 and validation RMSE is 2.1138878886870676


For rp/rank (0.6,18) the training RMSE is 0.7303739363868421 and validation RMSE is 2.09519140800299


For rp/rank (0.6,22) the training RMSE is 0.7303515297098322 and validation RMSE is 2.0666486100990786
The best model was trained with rp/rank (0.6,22,als_5ae04a39b092)


import org.apache.spark.ml.recommendation.{ALS, ALSModel}

import org.apache.spark.ml.evaluation.RegressionEvaluator
// old API: https://spark.apache.org/docs/2.1.0/api/scala/index.html#org.apache.spark.mllib.recommendation.ALS
// new API: https://spark.apache.org/docs/2.1.0/api/scala/index.html#org.apache.spark.ml.recommendation.ALS


seed: Int = 5
niters: Int = 10
regularization_parameters: Array[Double] = Array(0.3, 0.6)
ranks: Array[Int] = Array(18, 22)
tolerance: Double = 0.02
defined function rmse
minerr: Double = 2.0666486100990786
best: (Double, Int, ml.recommendation.ALSModel) = (0.6, 22, als_5ae04a39b092)

In [10]:
val err = rmse(best._3, test)._1
println(s"For testing data the RMSE is $err") // 2.0700

[Stage 513:====================================================>(199 + 1) / 200]

For testing data the RMSE is 2.070002363103332

err: Double = 2.070002363103332